# Importing Libraries 

In [1]:
import pandas as pd
import numpy as np
import sys

### Importing Data as pandas dataframe and preprocessing it

In [85]:
df = pd.read_csv(r"C:\Users\Aman-Verma\Downloads\ASP Transition Matrix and HMM\RP Data\NIFTY 2010-20.csv",error_bad_lines=False)

In [86]:
df = df.drop('High',axis=1)
df = df.drop('Low',axis=1)
df = df.drop('Adj Close',axis=1)
df = df.drop('Volume',axis=1)

In [87]:
df.describe()

,Open,Close
count,2445.000000,2445.000000
mean,7758.714319,7752.939513
std,2212.016988,2208.878781
min,4623.149902,4544.200195
25%,5681.100098,5677.899902
50%,7788.500000,7784.649902
75%,9657.150391,9657.549805
max,12274.900391,12271.799805


In [88]:
df.head()

,Date,Open,Close
0,2010-01-04,5200.899902,5232.200195
1,2010-01-05,5277.149902,5277.899902
2,2010-01-06,5278.149902,5281.799805
3,2010-01-07,5281.799805,5263.100098
4,2010-01-08,5264.250000,5244.750000


### Setting up the parameters for the model 

In [89]:
df['Difference'] = df['Close'].diff(-1)

In [90]:
df['Change Mod'] = df['Difference'].diff(-1).abs() / df['Open']

In [91]:
df.head()

,Date,Open,Close,Difference,Change Mod
0,2010-01-04,5200.899902,5232.200195,-45.699707,0.008037
1,2010-01-05,5277.149902,5277.899902,-3.899903,0.004283
2,2010-01-06,5278.149902,5281.799805,18.699707,0.000066
3,2010-01-07,5281.799805,5263.100098,18.350098,0.004355
4,2010-01-08,5264.250000,5244.750000,-4.649902,0.008292


In [92]:
df.describe()

,Open,Close,Difference,Change Mod
count,2445.000000,2445.000000,2420.000000,2395.000000
mean,7758.714319,7752.939513,-2.810042,0.010043
std,2212.016988,2208.878781,71.635639,0.008587
min,4623.149902,4544.200195,-569.400390,0.000013
25%,5681.100098,5677.899902,-45.312500,0.003737
50%,7788.500000,7784.649902,-3.550049,0.007839
75%,9657.150391,9657.549805,37.762573,0.014107
max,12274.900391,12271.799805,490.950195,0.067745


### Setting up the range  

In [93]:
criteria = [df['Difference'].between(-1000, -df['Change Mod']*df['Open']/2), df['Difference'].between( -df['Change Mod']*df['Open']/2, df['Change Mod']*df['Open']/2), df['Difference'].between(df['Change Mod']*df['Open']/2,1000)]
values = ['Down', 'Stagnant', 'Up']

df['Movement'] = np.select(criteria, values, 'Stagnant')

In [94]:
df.head()

,Date,Open,Close,Difference,Change Mod,Movement
0,2010-01-04,5200.899902,5232.200195,-45.699707,0.008037,Down
1,2010-01-05,5277.149902,5277.899902,-3.899903,0.004283,Stagnant
2,2010-01-06,5278.149902,5281.799805,18.699707,0.000066,Up
3,2010-01-07,5281.799805,5263.100098,18.350098,0.004355,Up
4,2010-01-08,5264.250000,5244.750000,-4.649902,0.008292,Stagnant


### Showing number of States  

In [95]:
df['Movement'].value_counts()

Stagnant    925
Down        846
Up          698
Name: Movement, dtype: int64

In [96]:
z = list(df['Movement'])

### Transition Probability Matrix 

In [97]:
pd.crosstab(pd.Series(z[1:],name='Tomorrow'),
            pd.Series(z[:-1],name='Today'),normalize=0)

Today,Down,Stagnant,Up
Tomorrow,,,
Down,0.421302,0.431953,0.146746
Stagnant,0.394595,0.256216,0.349189
Up,0.179083,0.461318,0.359599


In [98]:
from itertools import islice

def window(seq, n=2):
    "Sliding window width n from seq.  From old itertools recipes."""
    it = iter(seq)
    result = tuple(islice(it, n))
    if len(result) == n:
        yield result
    for elem in it:
        result = result[1:] + (elem,)
        yield result

In [99]:
import pandas as pd

pairs = pd.DataFrame(window(z), columns=['Today', 'Tomorrow'])
counts = pairs.groupby('Today')['Tomorrow'].value_counts()
probs = (counts).unstack()

### Transition Count Matrix

In [100]:
probs

Tomorrow,Down,Stagnant,Up
Today,,,
Down,356,365,125
Stagnant,365,237,322
Up,124,323,251


# End